# Machine Learning Data Quality

## Load Data

In [1]:
import numpy as np

def generate_dataset(num_features, num_samples, anomaly_percent):
    """
    Generates a random dataset with a specified number of features, number of samples, and percentage of anomalies.

    Parameters:
    num_features (int): Number of features in the dataset.
    num_samples (int): Number of samples in the dataset.
    anomaly_percent (float): Percentage of anomalies in the dataset.

    Returns:
    X (numpy.ndarray): A 2D numpy array containing the features of the dataset.
    y (numpy.ndarray): A 1D numpy array containing the labels of the dataset.
    """

    # Generate random data
    X = np.random.rand(num_samples, num_features)

    # Generate random labels
    num_anomalies = int(num_samples * (anomaly_percent / 100))
    y = np.zeros(num_samples)
    y[:num_anomalies] = 1
    np.random.shuffle(y)

    return X, y


## Isolation Forest

## Evaluate

---